In [1]:
# To reload external files automatically (ex: utils)
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import numpy as np
import calendar as cld
import matplotlib.pyplot as plt
#import proplot as plot # New plot library (https://proplot.readthedocs.io/en/latest/)
#plot.rc['savefig.dpi'] = 300 # 1200 is too big! #https://proplot.readthedocs.io/en/latest/basics.html#Creating-figures
from scipy import stats
import xesmf as xe # For regridding (https://xesmf.readthedocs.io/en/latest/)

import glob
import os
import re


#import sys
# sys.path.insert(1, '/home/mlalande/notebooks/utils') # to include my util file in previous directory
#sys.path.insert(1, '../utils') # to include my util file in previous directory
#import utils as u # my personal functions
#u.check_python_version()
# u.check_virtual_memory()
#3.

In [2]:
# For parallelisation
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=4, threads_per_worker=3, memory_limit='6GB')
# client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 22.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45269,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 22.35 GiB
Comm: tcp://127.0.0.1:46359,Total threads: 3
Dashboard: http://127.0.0.1:44281/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:40023,


In [3]:
client.close()

In [30]:
##create folders (First time done)
WY_list = [str(year) for i, year in enumerate(range(1999, 2017))]
##MLWY_list = ['WY'+str(year)+'_'+str(i).zfill(2) for i, year in enumerate(range(1999, 2017))]
#path_out = '/bettik/PROJECTS/pr-regional-climate/santolam/HMA_SR_daily_7km/'

##!mkdir {path_out}
#for WY in WY_list:
#    print(WY)
#    !mkdir {path_out}{WY}


In [5]:
#WY0=[WY_list[0]][0]
#print(WY0)

1999


In [6]:
#WY_list[4][-2:]

In [18]:
#!ls '/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/' # | wc -l

HMA_SR_D_v01_N27_0E100_0_agg_16_WY1999_00.xml
HMA_SR_D_v01_N27_0E100_0_agg_16_WY1999_00_FORCING_POST.nc
HMA_SR_D_v01_N27_0E100_0_agg_16_WY1999_00_MASK.nc
HMA_SR_D_v01_N27_0E100_0_agg_16_WY1999_00_SD_POST.nc
HMA_SR_D_v01_N27_0E100_0_agg_16_WY1999_00_SNOW_ALBEDO.nc
HMA_SR_D_v01_N27_0E100_0_agg_16_WY1999_00_SWE_SCA_POST.nc
HMA_SR_D_v01_N27_0E66_0_agg_16_WY1999_00.xml
HMA_SR_D_v01_N27_0E66_0_agg_16_WY1999_00_FORCING_POST.nc
HMA_SR_D_v01_N27_0E66_0_agg_16_WY1999_00_MASK.nc
HMA_SR_D_v01_N27_0E66_0_agg_16_WY1999_00_SD_POST.nc
HMA_SR_D_v01_N27_0E66_0_agg_16_WY1999_00_SNOW_ALBEDO.nc
HMA_SR_D_v01_N27_0E66_0_agg_16_WY1999_00_SWE_SCA_POST.nc
HMA_SR_D_v01_N27_0E67_0_agg_16_WY1999_00.xml
HMA_SR_D_v01_N27_0E67_0_agg_16_WY1999_00_FORCING_POST.nc
HMA_SR_D_v01_N27_0E67_0_agg_16_WY1999_00_MASK.nc
HMA_SR_D_v01_N27_0E67_0_agg_16_WY1999_00_SD_POST.nc
HMA_SR_D_v01_N27_0E67_0_agg_16_WY1999_00_SNOW_ALBEDO.nc
HMA_SR_D_v01_N27_0E67_0_agg_16_WY1999_00_SWE_SCA_POST.nc
HMA_SR_D_v01_N27_0E83_0_agg_16_WY1999_00.xml
H

In [19]:
#i=0
#WY0=[WY_list[i]][0]
#print(WY0)
#WY1=[WY_list[i+1]][0][-2:]
#print(WY1)

1999
00


In [47]:
WY0=int(1999) #[WY_list[i]][0]
print(WY0)
WY1=str((WY0+1))[-2:]
print(WY1)


1999
00


In [20]:

path = '/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/'+str(WY0)+'/'
path_out = f'/bettik/PROJECTS/pr-regional-climate/santolam/HMA_SR_daily_7km_{domain}/'+WY0+'/'

list_files =glob.glob((path+f'HMA_SR_D_v01_N*_*E*_0_agg_16_WY{WY0}_{WY1}_SD_POST.nc'))

fileNames = [os.path.basename(f) for f in list_files]
#    for ds_name in ['SD_POST']:
#        print(ds_name)
#        list_files = [f for f in os.listdir(path) if ds_name in f]

#print(list_files[0:10])
print(fileNames[0:10])
#print(len(list_files))

['HMA_SR_D_v01_N34_0E89_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N37_0E98_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N35_0E91_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N33_0E96_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N39_0E92_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N36_0E97_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N33_0E91_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N43_0E82_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N37_0E71_0_agg_16_WY1999_00_SD_POST.nc', 'HMA_SR_D_v01_N33_0E64_0_agg_16_WY1999_00_SD_POST.nc']


In [ ]:
print(WY)

In [21]:
##Selecting subdomain range, year and taking only SD_Post and Mask
year=WY0
# base directory
data_dir = f"/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/{year}/"

# define lat/lon selection range
lat_min, lat_max = 35, 43
lon_min, lon_max = 66, 79

# regex to extract N## and E### from filenames
pattern = re.compile(r"N(\d+)_0E(\d+)_")

# lists to store filepaths
sd_post_files = []
mask_files = []

# loop through directory files
for fname in os.listdir(data_dir):
    match = pattern.search(fname)
    if match:
        lat = int(match.group(1))
        lon = int(match.group(2))
        if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
            fullpath = os.path.join(data_dir, fname)
            if fname.endswith("_SD_POST.nc"):
                sd_post_files.append(fullpath)
            elif fname.endswith("_MASK.nc"):
                mask_files.append(fullpath)

# sort alphabetically (optional, but helps align lat/lon ordering)
sd_post_files.sort()
mask_files.sort()

print("SD_POST files:")
for f in sd_post_files:
    print(f)

print("\nMASK files:")
for f in mask_files:
    print(f)


SD_POST files:
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E67_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E68_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E69_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E70_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E71_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E72_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E74_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E75_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E76_0_agg_16_WY1999_00_SD_POS

In [42]:
print(sd_post_files[0])
print(mask_files[0])
print(len(sd_post_files))

/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E67_0_agg_16_WY1999_00_SD_POST.nc
/bettik/PROJECTS/pr-regional-climate/inputs/HMA_SR_D/1999/HMA_SR_D_v01_N35_0E67_0_agg_16_WY1999_00_MASK.nc
96


In [43]:
#ds_test=xr.open_dataset(sd_post_files[0])['SD_Post']
#ds_mask_test=xr.open_dataset(mask_files[0])['Non_seasonal_snow_mask']
#ds_new_test=ds_test.isel(Stats=0).where(ds_mask_test== 0).coarsen(Latitude=15, Longitude=15).mean()
#print(ds_new_test)
#fileName_new= sd_post_files[0].split('/'+str(year)+'/')[1].split('.nc')[0]+ '_MASKED_coarsen15x15.nc'
##path = '/mnt/lalandmi/equipes/C2H/HMASR/HMA_SR_D/'+WY+'/'
#path_out = '/bettik/PROJECTS/pr-regional-climate/santolam/HMA_SR_daily_7km/'+str(year)+'/'
#    
#ds_new_test.to_netcdf(path_out+fileName_new)

In [45]:
for k in range(len(sd_post_files)):
    print(k)
    ds_test=xr.open_dataset(sd_post_files[k])['SD_Post']
    ds_mask_test=xr.open_dataset(mask_files[k])['Non_seasonal_snow_mask']
    ds_new_test=ds_test.isel(Stats=0).where(ds_mask_test== 0).coarsen(Latitude=15, Longitude=15).mean()
#print(ds_new_test)
    fileName_new= sd_post_files[k].split('/'+str(year)+'/')[1].split('.nc')[0]+ '_MASKED_coarsen15x15.nc'
#path = '/mnt/lalandmi/equipes/C2H/HMASR/HMA_SR_D/'+WY+'/'
    path_out = f'/bettik/PROJECTS/pr-regional-climate/santolam/HMA_SR_daily_7km_{domain}/{year}/'
    ds_new_test.to_netcdf(path_out+fileName_new)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
